In [197]:
import pandas as pd
import numpy as np
import os
import openpyxl 

In [198]:
import pandas as pd

def excel_to_df(excel_file: str, sheet_name: str = None) -> pd.DataFrame:

    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    df.columns = (
        df.columns
        .str.lower()
        .str.strip()
        .str.replace(" ", "_")
        .str.replace(r"[^0-9a-zA-Z_]", "", regex=True)
    )
    
    df = df.convert_dtypes()
    
    return df

In [199]:
xl_df = excel_to_df(r"C:\Users\arypn\OneDrive\Desktop\CS307\MEHOINUSA672N.xlsx", sheet_name="Annual")

xl_df.head()


,observation_date,mehoinusa672n
0,1984-01-01,60420
1,1985-01-01,61570
2,1986-01-01,63850
3,1987-01-01,64650
4,1988-01-01,65130


In [200]:
csv_df = pd.read_csv(r"C:\Users\arypn\OneDrive\Desktop\CS307\Real_Estate_Sales_2001-2023_GL.csv", encoding="utf-8")
csv_df.head(3)

C:\Users\arypn\AppData\Local\Temp\ipykernel_45232\3174339068.py:1: DtypeWarning: Columns (7,8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  csv_df = pd.read_csv(r"C:\Users\arypn\OneDrive\Desktop\CS307\Real_Estate_Sales_2001-2023_GL.csv", encoding="utf-8")


,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,04/14/2021,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,05/26/2021,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN
2,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.463,Commercial,NaN,NaN,NaN,NaN,NaN


In [201]:
xl_df.columns

Index(['observation_date', 'mehoinusa672n'], dtype='object')

In [202]:
csv_df.columns

Index(['Serial Number', 'List Year', 'Date Recorded', 'Town', 'Address',
       'Assessed Value', 'Sale Amount', 'Sales Ratio', 'Property Type',
       'Residential Type', 'Non Use Code', 'Assessor Remarks', 'OPM remarks',
       'Location'],
      dtype='object')

In [203]:
csv_df["Date Recorded"] = pd.to_datetime(csv_df["Date Recorded"], format="%m/%d/%Y", errors="coerce")
xl_df["observation_date"] = pd.to_datetime(xl_df["observation_date"], errors="coerce")
xl_df["Year"] = xl_df["observation_date"].dt.year
csv_df = csv_df.rename(columns={"List Year": "Year"})

In [204]:
csv_df.head(2)

,Serial Number,Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,2021-04-14,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,2021-05-26,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN


In [205]:
merged_df = csv_df.merge(
    xl_df[["Year", "mehoinusa672n"]],
    on="Year",
    how="left"
)

In [206]:
merged_df.head(4)

,Serial Number,Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location,mehoinusa672n
0,2020177,2020,2021-04-14,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014),81580
1,2020225,2020,2021-05-26,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN,81580
2,2020348,2020,2021-09-13,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.463,Commercial,NaN,NaN,NaN,NaN,NaN,81580
3,2020090,2020,2020-12-14,Ansonia,57 PLATT ST,127400.0,202500.0,0.6291,Residential,Two Family,NaN,NaN,NaN,NaN,81580


In [207]:
# Since we are mostly focusing on pre an during covid, we will be focusing only on 2015 to 2023

In [208]:
merged_df["Date"] = merged_df["Date Recorded"]
merged_df = merged_df.drop(columns=["Non Use Code", "Assessor Remarks", "OPM remarks", "Location", "Date Recorded"])
merged_df = merged_df.sort_values("Year")

In [209]:
filtered_df = merged_df[(merged_df["Year"] >= 2015) & (merged_df["Year"] <= 2023)]

In [210]:
filtered_df.head()

,Serial Number,Year,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,mehoinusa672n,Date
855309,150386,2015,Southbury,147 A HERITAGE VILLAGE,58300.0,77000.0,0.757143,Condo,Condo,72790,2016-07-01
855308,150019,2015,Portland,4 PENNY CORNER ROAD,154140.0,210000.0,0.734,Single Family,Single Family,72790,2015-10-29
855307,150149,2015,Thompson,1158 RIVERSIDE DR,172300.0,198000.0,0.870202,Single Family,Single Family,72790,2016-07-06
855306,150108,2015,Plainville,69 NORTHWEST DR F-47,114450.0,143000.0,0.80035,Condo,Condo,72790,2016-01-19
855305,150081,2015,Stonington,29 WILBUR HILL RD,417200.0,590000.0,0.707119,Single Family,Single Family,72790,2015-12-23


In [211]:
filtered_df.shape

(462879, 11)

In [212]:
missing_rows = filtered_df[filtered_df.isna().any(axis=1)]
print(missing_rows.shape)
missing_rows.head(2)

(41212, 11)


,Serial Number,Year,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,mehoinusa672n,Date
125439,150145,2015,Portland,182 MAIN ST,715260.0,717149.0,0.997366,NaN,NaN,72790,2016-07-15
125438,15122,2015,North Haven,2 BLUE HILLS ROAD,102550.0,125000.0,0.8204,NaN,NaN,72790,2016-01-05


In [213]:
filtered_df.isna().any()

Serial Number       False
Year                False
Town                False
Address              True
Assessed Value      False
Sale Amount         False
Sales Ratio         False
Property Type        True
Residential Type     True
mehoinusa672n       False
Date                 True
dtype: bool

In [214]:
print(filtered_df[filtered_df["Property Type"].isna() & filtered_df["Residential Type"].isna()].shape)
filtered_df[filtered_df["Property Type"].isna() & filtered_df["Residential Type"].isna()].head(2)

(20525, 11)


,Serial Number,Year,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,mehoinusa672n,Date
125439,150145,2015,Portland,182 MAIN ST,715260.0,717149.0,0.997366,NaN,NaN,72790,2016-07-15
125438,15122,2015,North Haven,2 BLUE HILLS ROAD,102550.0,125000.0,0.8204,NaN,NaN,72790,2016-01-05


In [215]:
Cleaned_Df = filtered_df.dropna(subset=["Property Type", "Residential Type"], how="all")

In [216]:
print(Cleaned_Df.shape)
Cleaned_Df.head(2)

(442354, 11)


,Serial Number,Year,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,mehoinusa672n,Date
855309,150386,2015,Southbury,147 A HERITAGE VILLAGE,58300.0,77000.0,0.757143,Condo,Condo,72790,2016-07-01
855308,150019,2015,Portland,4 PENNY CORNER ROAD,154140.0,210000.0,0.734,Single Family,Single Family,72790,2015-10-29


In [225]:
cpi_dict = {
    2015: 237.017,
    2016: 240.007,
    2017: 245.120,
    2018: 251.107,
    2019: 255.657,
    2020: 258.811,
    2021: 270.970,
    2022: 292.655,
    2023: 305.109
}

import pandas as pd

cpi_df = pd.DataFrame(list(cpi_dict.items()), columns=['year', 'cpi'])


In [226]:
adj_df = Cleaned_Df.merge(cpi_df, left_on='Year', right_on='year', how='left')


In [227]:
cpi_2023 = cpi_dict[2023]

adj_df['price_2023'] = adj_df['Sale Amount'] * (cpi_2023 / adj_df['cpi'])



In [228]:
mean_price_year = (
    adj_df.groupby('Year')['price_2023']
          .mean()
          .reset_index()
          .rename(columns={'price_2023': 'mean_residential_price_2023usd'})
)



In [230]:
adj_df = adj_df.merge(mean_price_year, on='Year', how='left')
adj_df.shape

(442354, 16)

In [232]:
# Create final year-level dataset
final_year_df = (
    adj_df.groupby('Year')
          .agg({
              'price_2023': 'mean',            # mean inflation-adjusted home price
              'Sale Amount': 'mean',           # mean nominal (raw) home price
              'mehoinusa672n': 'mean'          # median household income
          })
          .reset_index()
)

final_year_df = final_year_df.rename(columns={
    'price_2023': 'mean_residential_price_2023usd',
    'Sale Amount': 'mean_raw_sale_amount',
    'mehoinusa672n': 'median_household_income'
})

final_year_df


,Year,mean_residential_price_2023usd,mean_raw_sale_amount,median_household_income
0,2015,366472.458068,284685.809313,72790.0
1,2016,496265.109321,390375.571002,75380.0
2,2017,431304.933886,346503.922841,76710.0
3,2018,384485.958065,316434.833033,77700.0
4,2019,448879.236188,376124.987745,83260.0
5,2020,713194.454130,604972.550360,81580.0
6,2021,604627.690901,536975.197072,81270.0
7,2022,581484.881788,557749.715937,79500.0
8,2023,538314.328200,538314.328200,82690.0
